# Natural Language Processing: Week 2 - Word Embeddings

워드 임베딩(Word Embedding)은 단어를 벡터로 표현하는 방법으로, One-Hot 형태의 Sparse Vector가 아닌 Dense Vector 형태로 나타내는 과정이다.

이 밀집 벡터를 워드 임베딩 과정을 통해 나온 결과라고 하여 임베딩 벡터(embedding vector)라고도 한다.



머신 러닝에서 텍스트 분류. 그 중에서도 특히, 감성 분류를 연습하기 위해 자주 사용하는 데이터에는 영화 사이트 IMDB의 리뷰 데이터가 있다. 

이 데이터는 리뷰에 대한 텍스트와 해당 리뷰가 긍정인 경우 1을 부정인 경우 0으로 표시한 레이블로 구성된 데이터이다.

스탠포드 대학교에서 2011년에 낸 논문에서 이 데이터를 소개하였으며, 당시 논문에서는 이 데이터를 훈련 데이터와 테스트 데이터를 50:50대 비율로 분할하여 88.89%의 정확도를 얻었다고 소개하고 있다.

논문 링크 : http://ai.stanford.edu/~amaas/papers/wvSent_acl2011.pdf

텐서플로우에서는 해당 데이터를 바로 받아볼 수 있도록 지원하고 있으며, 이를 이용하여 실습을 진행해보도록 하겠다.

In [5]:
import numpy as np
import tensorflow as tf
tf.__version__
# 본인의 텐서플로우 버전이 1.X인 경우 다음을 실행하자
# tf.enable_eager_execution()

'2.6.0'

## Load Dataset

해당 데이터는 25,000개의 train sample과 25,000개의 test sample로 나뉜다

In [3]:
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteX8TPIC/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteX8TPIC/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
# Split Dataset
train_data, test_data = imdb['train'], imdb['test']

In [11]:
# Converting

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s, l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [12]:
# labels to Numpy array
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [14]:
# define 
vocab_size = 10_000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_token = '<OOV>'

# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_length)

In [15]:
# define Neural Network

model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')
])

## How can we use vectors?

현재 우리의 데이터셋은 해당 리뷰의 긍, 부정을 레이블로 나타내고 있다. 즉, 감성 분석의 답변을 제공하고 있는데, 이때 부정적인 리뷰라고 레이블되어 있는 sentence에서 자주 등장하는 단어는 부정적인 감정을 지녔다고 판단되게 된다.

비슷한 감정을 지닌 단어들의 벡터는 유사한 경향을 보이게 되며, 이는 Neural Network 학습에서 이용되게 된다.

In [16]:
# Model Compile & Summary
model.compile(
    loss='binary_crossentropy', 
    optimizer = 'adam', 
    metrics = ['accuracy']
    )

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [17]:
num_epochs = 10
model.fit(padded,
          training_labels_final,
          epochs = num_epochs,
          validation_data = (testing_padded, testing_labels_final)
          )

Epoch 1/10
782/782 [==============================] - 6s 6ms/step - loss: 0.4957 - accuracy: 0.7402 - val_loss: 0.3486 - val_accuracy: 0.8471
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2397 - accuracy: 0.9073 - val_loss: 0.3696 - val_accuracy: 0.8400
Epoch 3/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0940 - accuracy: 0.9765 - val_loss: 0.4423 - val_accuracy: 0.8274
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0250 - accuracy: 0.9968 - val_loss: 0.5210 - val_accuracy: 0.8246
Epoch 5/10
782/782 [==============================] - 4s 6ms/step - loss: 0.0073 - accuracy: 0.9992 - val_loss: 0.5821 - val_accuracy: 0.8241
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.6350 - val_accuracy: 0.8268
Epoch 7/10
782/782 [==============================] - 4s 6ms/step - loss: 9.5330e-04 - accuracy: 1.0000 - val_loss: 0.6808 - val_accuracy: 0.8273
Ep

In [18]:
e = model.layers[0]
weights = e.get_weights()[0]

print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


## Write the vectors and their metadata auto files

In [27]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
  return " ".join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[0]))
print(training_sentences[0])

? ? b this was an absolutely terrible movie don't be <OOV> in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining <OOV> like christopher <OOV> good name i could barely sit through it
b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pa

In [24]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + '\n')
  out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')

out_v.close()
out_m.close()

In [26]:
# import files

try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

[TensorFlow projector](https://projector.tensorflow.org)